In [45]:
import pandas as pd
import os
import google.auth
import numpy as np

In [20]:
credentials, project_id = google.auth.default()
rs_dataset_project = 'som-nero-nigam-starr'
rs_dataset = 'jlemmon_explore'
rs_table = 'sudden_cardiac_death_cohort'
cohort_fpath = '/local-scratch/nigam/projects/jlemmon/cl-clmbr/experiments/main/data/'

/local-scratch/nigam/envs/jlemmon/conl/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [18]:
query = f'SELECT * FROM {rs_dataset_project}.{rs_dataset}.{rs_table}'

scd_cohort_df = pd.read_gbq(query, project_id=rs_dataset_project, dialect='standard', credentials=credentials)

In [19]:
print(scd_cohort_df)

      cohort_definition_id  person_id  start_date    end_date
0                        1   30313699  2009-12-14  2009-12-23
1                        1   30012248  2010-08-27  2010-09-03
2                        1   32477186  2014-02-27  2014-03-06
3                        1   30966573  2014-11-10  2014-11-17
4                        1   32168272  2016-12-17  2016-12-24
...                    ...        ...         ...         ...
4036                     1   32443919  2016-12-16  2016-12-23
4037                     1   31050037  2017-08-29  2017-09-05
4038                     1   32101654  2019-01-23  2019-01-31
4039                     1   30165733  2019-10-06  2019-10-07
4040                     1   42130121  2019-10-06  2019-10-13

[4041 rows x 4 columns]


In [21]:
def read_file(filename, columns=None, **kwargs):
    print(filename)
    load_extension = os.path.splitext(filename)[-1]
    if load_extension == ".parquet":
        return pd.read_parquet(filename, columns=columns,**kwargs)
    elif load_extension == ".csv":
        return pd.read_csv(filename, usecols=columns, **kwargs)

In [79]:
cohort = read_file(
    os.path.join(
        cohort_fpath,
        "cohort/cohort.parquet"
    ),
    engine='pyarrow'
)

/local-scratch/nigam/projects/jlemmon/cl-clmbr/experiments/main/data/cohort/cohort.parquet


In [80]:
cohort['sudden_cardiac_death'] = 0
print(cohort.head())

   person_id          admit_date      discharge_date admit_date_midnight  \
0   29936887 2019-12-22 22:44:00 2019-12-26 16:00:00 2019-12-22 23:59:00   
1   29936888 2010-07-12 06:34:00 2010-07-14 10:38:00 2010-07-12 23:59:00   
2   29936900 2014-10-31 15:08:00 2014-11-03 13:25:00 2014-10-31 23:59:00   
3   29936914 2018-06-23 05:20:00 2018-06-25 13:48:00 2018-06-23 23:59:00   
4   29936936 2018-02-08 12:05:00 2018-02-10 16:30:00 2018-02-08 23:59:00   

  discharge_date_midnight  hospital_mortality death_date  month_mortality  \
0     2019-12-26 23:59:00                   0        NaT                0   
1     2010-07-14 23:59:00                   0        NaT                0   
2     2014-11-03 23:59:00                   0        NaT                0   
3     2018-06-25 23:59:00                   0        NaT                0   
4     2018-02-10 23:59:00                   0        NaT                0   

   LOS_days  LOS_7  ...               race_eth_raw  \
0         4      0  ...  B

In [62]:
cohort_pat_ids = list(cohort['person_id'])
scd_pat_ids = list(scd_cohort_df['person_id'])

In [63]:
print(len(cohort_pat_ids))
print(len(scd_pat_ids))
print(len(set(scd_pat_ids)))

224137
4041
3985


In [73]:
cohort.loc[cohort.person_id.isin(scd_pat_ids), 'sudden_cardiac_death'] = 1

/tmp/ipykernel_1933659/1821248294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cohort['sudden_cardiac_death'][cohort.person_id.isin(scd_pat_ids)] = 1


In [81]:
cohort['sudden_cardiac_death'] = cohort['sudden_cardiac_death'].mask(cohort.person_id.isin(scd_pat_ids), 1)

In [83]:
print(cohort.head())

   person_id          admit_date      discharge_date admit_date_midnight  \
0   29936887 2019-12-22 22:44:00 2019-12-26 16:00:00 2019-12-22 23:59:00   
1   29936888 2010-07-12 06:34:00 2010-07-14 10:38:00 2010-07-12 23:59:00   
2   29936900 2014-10-31 15:08:00 2014-11-03 13:25:00 2014-10-31 23:59:00   
3   29936914 2018-06-23 05:20:00 2018-06-25 13:48:00 2018-06-23 23:59:00   
4   29936936 2018-02-08 12:05:00 2018-02-10 16:30:00 2018-02-08 23:59:00   

  discharge_date_midnight  hospital_mortality death_date  month_mortality  \
0     2019-12-26 23:59:00                   0        NaT                0   
1     2010-07-14 23:59:00                   0        NaT                0   
2     2014-11-03 23:59:00                   0        NaT                0   
3     2018-06-25 23:59:00                   0        NaT                0   
4     2018-02-10 23:59:00                   0        NaT                0   

   LOS_days  LOS_7  ...               race_eth_raw  \
0         4      0  ...  B